# Interleaving Simulation and Steering
We use a batch strategy [in our previous example](./molecular-design-with-parsl.ipynb) that, while simple to implement, leads to under-utilization.
The core problem of a batch strategy is that only one type of task - simulation, training, or inference - at a single time.
The serial nature results in several points during the workflow where either there are not enough tasks (e.g., one model to train)
or tail-down loses while we wait for the last tasks from a batch to complete before starting the next type.
This example shows how to increase parallelism by using [Colmena](https://colmena.readthedocs.io/en/latest/) to run multiple kinds of tasks concurrently.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from colmena.models import Result
from colmena.task_server.parsl import ParslTaskServer
from colmena.redis.queue import make_queue_pairs
from colmena.thinker.resources import ResourceCounter
from colmena.thinker import BaseThinker, event_responder, task_submitter, result_processor
from parsl.executors import HighThroughputExecutor
from parsl.config import Config
from random import shuffle
from threading import Lock
from typing import List
from chemfunctions import compute_vertical, train_model, run_model
from tqdm.notebook import tqdm
import pandas as pd

## Load in the Data
We're going to use the same problem as the previous example.

In [2]:
search_space = pd.read_csv('data/QM9-search.tsv', delim_whitespace=True).sample(1024)  # Our search space of molecules

In [3]:
initial_count: int = 8  # Number of calculations to run at first

In [4]:
search_count: int = 16   # Number of molecules to evaluate in total

In [5]:
batch_size: int = 4  # Number of molecules to evaluate in each batch of simulations

## Configuring the Task Server
Colmena applications have three parts: a _Task Server_ that performs work at the direction of a _Thinker_ through a _Task Queue_.

### Creating Task Queue
A task queue is responsible for conveying requests to perform a computation to a Task Server, and then supplying results back to the Thinker.
Creating a task queue requires defining connection information to Redis and the names of separate topics used to separate different kinds of tasks

In [6]:
client_queues, server_queues = make_queue_pairs(hostname='localhost', topics=['simulate', 'train', 'infer'], serialization_method='pickle')

### Defining Task Server
The Task Server requires a task queue to communicate through, a list of methods, and a set of computational resources to run them on. (See [Colmena Docs](https://colmena.readthedocs.io/en/latest/how-to.html#configuring-a-task-server))

The computation resources are defined using Parsl's definitions. We'll use the same one as the previous example.

In [7]:
config = Config(
    executors=[HighThroughputExecutor(
        max_workers=2, # Allows a maximum of two workers
        cpu_affinity='block' # Prevents workers from using the same cores
    )]
)

We supply a list of Python functions to define the methods and also give the constructor a link to the queues.

In [8]:
task_server = ParslTaskServer(
    methods=[compute_vertical, train_model, run_model],
    queues=server_queues,
    config=config,
)

The task server runs in the background. 
> *NOTE*: You must kill it before exiting the notebook by sending a kill signal

In [9]:
task_server.start()

The server will run tasks on request from a queue and send them back on a different Redis queue.
The client queue object provides a `send_inputs` and `get_result` method to perform these operations.

In [10]:
%%time
client_queues.send_inputs('C', method='compute_vertical')
result = client_queues.get_result()

CPU times: user 0 ns, sys: 3.35 ms, total: 3.35 ms
Wall time: 3.64 s


Both accept a "topic" option that allows for multiplexing.

In [11]:
client_queues.send_inputs('C', method='compute_vertical', topic='simulate')

# Show that we do not pull results on other topics
result = client_queues.get_result(topic='infer', timeout=15)
assert result is None  # None means a timeout occurred

# Pull from the correct queue
result = client_queues.get_result(topic='simulate')

Shut down the system for now

## Building a Thinker
The Thinker part of a Colmena application coordinates what tasks are run by the Task Server.

Thinker applications are built using a collection of threads ("agents") that cooperate to perform some task. 
For example, you can have an agent that records a simulation being completed and launches a second agent that manages retraining the models.

Below, we walk through how to build a thinker application though progressively more complex examples.

### Example 1: Simulating molecules in a predefined list
A steering policy in Colmena is defined through a [Thinker](https://colmena.readthedocs.io/en/latest/how-to.html#creating-a-thinker-application) class. 
The Thinker class has methods which can run as parallel threads and share information with each other via class attributes, 
which always includes a "resource allocation tracker" used to signal when resources are free.

A simple example for a Thinker is one that submits a new calculation from a list when another completes.

In [12]:
class ExampleThinker(BaseThinker):
    
    def __init__(self, queues, n_to_evaluate: int, n_parallel: int, 
                 molecule_list: List[str]):
        """Initialize the thinker
        
        Args:
            queues: Client side of queues
            n_to_evaluate: Number of molecules to evaluate
            n_parallel: Number of computations to run in parallel
            molecule_list: List of SMILES strings
        """
        super().__init__(
            queues, 
            ResourceCounter(n_parallel, ['simulate', 'train', 'infer'])
        )
        
        # Store the user settings
        self.molecule_list = set(molecule_list)
        self.n_to_evaluate = n_to_evaluate
        
        # Create a database of evaluated molecules
        self.database = dict()
        
        # Create a record of completed calculations
        self.computations = []
        
        # Create a priority list of molecules, starting with them ordered randomly
        self.priority_list = list(self.molecule_list)
        shuffle(self.priority_list)
        self.priority_list_lock = Lock()  # Ensures two agents cannot use it 
        
        # Create a tracker for how many sent and how many complete
        self.rec_progbar = tqdm(total=n_to_evaluate, desc='started')
        self.sent_progbar = tqdm(total=n_to_evaluate, desc='successful')
        
        # Assign all of the resources over to simulation
        self.rec.reallocate(None, 'simulate', n_parallel)
        
    @task_submitter(task_type='simulate', n_slots=1)
    def submit_calc(self):
        """Submit a calculation when resources are available"""
        
        with self.priority_list_lock:
            next_mol = self.priority_list.pop()  # Get the next best molecule
        
        # Send it to the task server to run
        self.queues.send_inputs(next_mol, method='compute_vertical')
        self.rec_progbar.update(1)
        
    @result_processor
    def receive_calc(self, result: Result):
        """Store the output of a run if it is successful"""
        
        # Mark that the resources are now free
        self.rec.release('simulate', 1)
        
        # Store the result if successful
        if result.success:
            # Store the result in a database
            self.database[result.args[0]] = result.value
            
            # Mark that we've received a result
            self.sent_progbar.update(1)
            
            # If we've got all of the simulations complete, stop
            if len(self.database) >= self.n_to_evaluate:
                self.done.set()
            
        # Store the result object for later processing
        self.computations.append(result)

We instantiate a copy of this thinker with the settings we want and then call `run` to start it working

In [13]:
thinker = ExampleThinker(client_queues, search_count, 2, search_space['smiles'].values)
thinker.run()

started:   0%|          | 0/16 [00:00<?, ?it/s]

successful:   0%|          | 0/16 [00:00<?, ?it/s]

Watch how the thinker only start new calculations after another one finishes. 
The ability to throttle will be important when we don't know which calculations to submit next until others have finished.

> The thinker will receive more than the requested number of calculations, as we stop submitting only after enough have completed and wait until all submitted tasks complete.

## Wrap up
Once complete, we send a "kill" signal to shutdown the task server. The task server will clean up any computational resources being used, then exit.

In [14]:
client_queues.send_kill_signal()
task_server.join()
print(f'Process exited with {task_server.exitcode} code')

Process exited with 0 code
